<h3>1.	Django signals execute synchronously by default. In Django, signal handler and sender are executed synchronously on the same thread. That is all blocking further execution until all handlers are completed.</h3>

In [20]:
from django.dispatch import Signal, receiver
import time

signal = Signal()

@receiver(signal)
def my_handler(sender, **kwargs):  
    print("Handler started\nWaiting 2 secs")
    time.sleep(2)  
    print("Handler finished")

# Sending the signal
print("Sending signal...")
signal.send(sender=None)  

print("Signal sent!")


Sending signal...
Handler started
Waiting 2 secs
Handler finished
Signal sent!


<h3>2.	Signals sent by Django operate within the thread that sent them. This implies that the code handling the signal executes directly there, within the same thread, whenever a signal is activated. The signal handler will halt all other thread activity until it completes.</h3>

In [21]:
import threading
from django.dispatch import Signal, receiver
import time

signal = Signal()

@receiver(signal)
def my_handler(sender, **kwargs):
    print("Handler started in thread.")
    time.sleep(3)
    print("Handler finished")

def send_signal():
    print("Sending signal from thread")
    signal.send(sender=None)
    print("Signal sent!")

# Start a new thread to send the signal
thread = threading.Thread(target=send_signal)
thread.start()
thread.join()  # Wait for the thread to finish


Sending signal from thread
Handler started in thread.
Handler finished
Signal sent!


3. <h4>Django signals run in the same database transaction as the caller. This means that any changes made to the database within the signal handler will be part of the same transaction and will either be committed or rolled back together with the changes made by the caller. If any error occurs during the execution  the entire transaction will be rolled back.</h4>

In [24]:
from django.db import transaction
from django.db import connection
from django.dispatch import Signal, receiver


user_created = Signal()

@receiver(user_created)
def log_user_creation(sender, **kwargs):
    
    with connection.cursor() as cursor:
        cursor.execute("INSERT INTO logs (message) VALUES (%s)", ['User created'])


def create_user(name):
    with transaction.atomic():  
        
        with connection.cursor() as cursor:
            cursor.execute("INSERT INTO users (name) VALUES (%s)", [name])

        
        user_created.send(sender=name)

try:
    create_user("DSAKMD SDAD")
    print("User created successfully.")
except Exception as e:
    print(f"Error occurred: {e}")


User created successfully.


In [25]:
class Rectangle:
    def __init__(self, length: int, width: int):
        self.length = length
        self.width = width

    def __iter__(self):
        yield {'length': self.length}
        yield {'width': self.width}
rect = Rectangle(6, 8)
for dim in rect:
    print(dim)


{'length': 6}
{'width': 8}
